In [1]:
import tensorflow as tf
import os
import _pickle as cPickle
import numpy as np

CIFAR_DIR = "C:\\Users\\i076453\\Downloads\\tech\\ml\\cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    """ read data from data file """
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.dataset
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)     # 不做filter, 将所有的数据都加载
            all_labels.append(labels)
        self._data = np.vstack(all_data)  # 纵向合并成
        self._data = self._data / 127.5 - 1 # 进行归一化 
            
        self._labels = np.hstack(all_labels) # 横向合并
        
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
        
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)] 
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

# batch_data, batch_labels = train_data.next_batch(10)
# print(batch_data)
# print(batch_labels)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
# 抽象残差连接块
def residual_block(x, output_channel):
    """residual connection implementation"""
    input_channel = x.get_shape().as_list()[-1]
    if input_channel * 2 == output_channel:
        increase_dim = True
        strides = (2, 2)  # 降采样
    elif input_channel == output_channel:
        increase_dim = False
        strides = (1,1) # 不降采样
    else:
        raise Exception("input channel cannnot match output channel")
    
    # 添加卷积层
    conv1 = tf.keras.layers.Conv2D(
                                output_channel,     # output channel number
                                (3,3),  # kernel size
                                strides = strides, # 步长
                                padding = 'same',  # 表示输出输入图像大小一致， 还有一个值是valid
                                activation=tf.nn.relu, # 激活函x数
                                name = 'conv1' # 名字
                                )(x)
    
    conv2 = tf.keras.layers.Conv2D(
                                output_channel,     # output channel number
                                (3,3),  # kernel size
                                strides = (1, 1), # 步长
                                padding = 'same',  # 表示输出输入图像大小一致， 还有一个值是valid
                                activation=tf.nn.relu, # 激活函x数
                                name = 'conv2' # 名字
                                )(conv1)

    
    if increase_dim:
        # [None, image_width, image_height, channel] -> [,,,channel x 2]
        pooled_x = tf.keras.layers.AveragePooling2D(
                                                (2,2),
                                                (2,2),
                                                padding='valid')(x)
        padded_x = tf.pad(pooled_x,
                          [[0,0],
                          [0,0],
                          [0,0],
                          [input_channel // 2, input_channel // 2]])
    else:
        padded_x = x
    output_x = conv2 + padded_x
    return output_x
    
    
def res_net(x, num_residual_blocks, num_filter_base, class_num):
    """residual network implementation"""
    """
    Args:
    - x: 
    - num_residual_blocks: eg: [3,4,6,3]
    - num_filter_base:
    - class_num: 
    """
    num_subsampling = len(num_residual_blocks)
    layers = []
    
    input_size = x.get_shape().as_list()[1:]
    with tf.variable_scope('conv0'):
        conv0 = tf.keras.layers.Conv2D(
                                num_filter_base,     # output channel number
                                (3,3),  # kernel size
                                strides = (1, 1), # 步长
                                padding = 'same', 
                                activation=tf.nn.relu, # 激活函x数
                                name = 'conv0' # 名字
                                )(x)
        layers.append(conv0)
    # eg: num_subsampling = 4, sample_id = [0,1,2,3]
    for sample_id in range(num_subsampling):
        for i in range(num_residual_blocks[sample_id]):
            with tf.variable_scope("conv%d_%d" % (sample_id, i)):
                conv = residual_block(layers[-1], num_filter_base * (2 ** sample_id))
                layers.append(conv)
    
    multiplier = 2 ** (num_subsampling - 1)
    assert layers[-1].get_shape().as_list()[1:] \
        == [input_size[0] / multiplier, 
            input_size[1] / multiplier,
            num_filter_base * multiplier]
    
    with tf.variable_scope('fc'):
        global_pool = tf.reduce_mean(layers[-1], [1,2])
        logtis = tf.keras.layers.Dense(class_num)(global_pool)
        layers.append(logtis)
    
    return layers[-1]
    
    
x = tf.placeholder(tf.float32, [None, 3072]) # data 的placeholder, None表示不确定的
# y的shape是 [0.6,5,3]
y = tf.placeholder(tf.int64, [None]) # labels的placeholder

# 转换成多通道
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 交换通道
# 32 x 32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1]) # [-1, 32, 32, 3]

# 
y_ = res_net(x_image, [2,3,2], 32, 10)


# 交叉熵
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# 它做了下面三件事
# y_ -> softmax
# y -> one hot
# loss = ylogy_

# bool 
predict = tf.argmax(y_, 1) # 在第二个维度求最大值

# [1,0,0,1,0,0,1,1...]
correct_predict = tf.equal(predict, y)

# 准确率
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float64))

# 有了loss目标函数和准确率后，再去做一个
# AdamOptimizer 是一个剔除向量的变种，1e-3就是10的-3次方0.001 
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [4]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict = {
                x: batch_data,
                y: batch_labels
            }
        )
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' \
                % (i+1, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size) 
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    }
                )
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' % (i+1, test_acc))

[Train] Step: 500, loss: 1.56996, acc: 0.45000
[Train] Step: 1000, loss: 2.15820, acc: 0.35000
[Train] Step: 1500, loss: 1.54940, acc: 0.40000
[Train] Step: 2000, loss: 0.99563, acc: 0.60000
[Train] Step: 2500, loss: 1.43976, acc: 0.50000
[Train] Step: 3000, loss: 1.08864, acc: 0.55000
[Train] Step: 3500, loss: 1.10997, acc: 0.70000
[Train] Step: 4000, loss: 0.67024, acc: 0.70000
[Train] Step: 4500, loss: 1.24659, acc: 0.60000
[Train] Step: 5000, loss: 0.74813, acc: 0.70000
(10000, 3072)
(10000,)
[Test] Step: 5000, acc: 0.65900
[Train] Step: 5500, loss: 0.69034, acc: 0.80000
[Train] Step: 6000, loss: 0.65254, acc: 0.70000
[Train] Step: 6500, loss: 0.93642, acc: 0.65000
[Train] Step: 7000, loss: 1.23133, acc: 0.70000
[Train] Step: 7500, loss: 1.03267, acc: 0.60000
[Train] Step: 8000, loss: 0.29113, acc: 0.80000
[Train] Step: 8500, loss: 0.56623, acc: 0.75000
[Train] Step: 9000, loss: 0.29022, acc: 0.95000
[Train] Step: 9500, loss: 0.75545, acc: 0.80000
[Train] Step: 10000, loss: 0.38396